In [190]:
#Packages needed
using Optim
using Plots
using StatsBase
using Distributions
using Statistics
using Random
using JuMP, Gurobi, GLPK
using TickTock
using Distributed
using CSV
using DataFrames
using JLD2
using Dates
####### This function just checks the OS and helps automate things instead of needed to copy and paste paths
function checkOS()
    path = pwd()
    if path[1] == '/'
        OS = "Mac"
    else
        OS = "Windows"
    end
    return OS
end
OS = checkOS()
file_path = pwd();
#####Load correct .jl files for functions (accommodates for Mac or Windows)
if OS == "Mac"
    inipath = file_path[1:findlast("Julia_Code",file_path)[end]]*"/"
    include(inipath * "1. Tradeoff Curvers and Value of AM Addition/2. Scripts/OptEvalCombined.jl")
else 
    inipath = file_path[1:findlast("Julia_Code",file_path)[end]] * "\\"
    include(inipath * "1. Tradeoff Curvers and Value of AM Addition\\2. Scripts\\OptEvalCombined.jl")
end

full_run_varied_dem_inputs (generic function with 2 methods)

In [191]:
#Starting String to help organize
starter = "136Iso_36_18_AMCapRatio_varyCV2_0.5xLT_run_"
"""
#for vary ADI
adi = collect(range(1,5,length=17))
cv2 = repeat([2],17)
description = []
for i = 1:17
    push!(description, starter * string(i))
end

"""
#for vary CV2
adi = repeat([1],20)
cv2 = collect(range(0.25,5, length=20))
description = []
for i = 1:20
    push!(description, starter * string(i))
end


In [192]:
###########Test Conditions
avg_dem = 18
lev = avg_dem .* adi
T = 60      #Time Horizon
G = 1 #Number of products
obs = 180
nreal = 100 # number of realizations to generate
LT = [7 7 4]
N=1000 # Number of replications to generate LTD based on
AM_capacity = 36

36

In [193]:
############Supply Chain Design
"""
#Kyle's paper matrix 1-3-2

trans_matrix = [0 25 25 25 0  0;
                0 0  0  0  25 0;
                0 0  0  0  50 50;
                0 0  0  0  0  25;
                0 0  0  0  0  0;
                0 0  0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6]


#Validation SC 1-1-4
trans_matrix = [0 25 0  0  0  0;
                0 0  25 25 25 25;
                0 0  0  0  0  0;
                0 0  0  0  0  0;
                0 0  0  0  0  0;
                0 0  0  0  0  0]

TM = [1]
DC = [2]
SL = [3 4 5 6]

#My network - 1-3-4 iso
trans_matrix = [0 25 25 25  0 0  0  0;
                0 0  0  0  25 25 0  0;
                0 0  0  0  25 25 0  0;
                0 0  0  0  0  0  25 25;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6 7 8]

#My network - 1-3-4 iso - increase in trans cost
trans_matrix = [0 25 25 50  0 0  0  0;
                0 0  0  0  25 25 0  0;
                0 0  0  0  25 25 0  0;
                0 0  0  0  0  0  25 25;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0;
                0 0  0  0  0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6 7 8]
"""

#Network - 1-3-6 Iso
trans_matrix = [0 25 25 25  0  0  0  0  0  0; #will vary the last 25 in this row
                0 0  0  0  25 25 25  0  0  0;
                0 0  0  0  0  25 25 25  0  0;
                0 0  0  0  0  0   0  0 25 25;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0;
                0 0  0  0  0  0   0  0  0  0]
TM = [1]
DC = [2 3 4]
SL = [5 6 7 8 9 10]

nsup, ncust, nTM, valid_TMrte, Suprte, CtTM, CtS = trans_matrix_math(trans_matrix,TM,DC,SL)

(3, 6, 1, [1.0; 1.0; 1.0], [1.0 1.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0], [25.0 25.0 25.0], [25.0 25.0 … 0.0 0.0; 0.0 25.0 … 0.0 0.0; 0.0 0.0 … 25.0 25.0])

In [194]:
Int(0b111000000)+1

449

In [195]:
####### This cell is to be run if you do not want to run the whole network (good to do when it is a larger network to reduce time)

#networks of interest for 134 network
"""
networks = [1; #no AM
            128; #Full AM
            113; #Centralized
            16; #Distributed
            17; #Iso DC
            4;  #Iso SLs
            20; #Iso Network Side
            13; #only Connected SLs have AM
            97; #Connected DCs have AM
            65; #DC1
            33; #DC2
            17; #DC3
            9; #SL1
            5; #SL2
            3; #SL3
            2] #SL4 
"""
#networks or interest for 136 network
networks = [1; #no AM
            512; #Full AM
            449; #Centralized
            64; #Distributed
            65; #Iso DC
            4;  #Iso SLs
            68; #Iso Network Side
            61; #only Connected SLs have AM
            385; #Connected DCs have AM
            445; #Connected Side
            257; #DC1
            129; #DC2
            65; #DC3
            9; #SL1
            5; #SL2
            3; #SL3
            2] #SL4 
"""
networks = [1; #no AM
            65; #Iso DC
            4; #Iso SLs
            68] #Iso network Side
"""

"networks = [1; #no AM\n            65; #Iso DC\n            4; #Iso SLs\n            68] #Iso network Side\n"

In [196]:
full_run_varied_dem_inputs(nsup, ncust, nTM, valid_TMrte, Suprte, CtTM, CtS, avg_dem, adi, cv2, lev, T, G, obs, nreal, LT, N, AM_capacity, description, networks)

idx_Croston = [1, 2, 3]
idx_SBA = Int64[]
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
Design 8
Design 9
Design 10
Design 11
Design 12
Design 13
Design 14
Design 15
Design 16
Design 17
Design 1
Design 2
Design 3
Design 4
Design 5
Design 6
Design 7
Design 8
Design 9
Design 10
Design 11
Design 12
Design 13
Design 14
Design 15
Design 16
Design 17
C:\Users\lmdoudni\Documents\Julia_Code\9. Raw Output Data\2021-10-18_136Iso_36_18_AMCapRatio_varyCV2_0.5xLT_run_1_1ADI_0_25CV2_18AvgDem.jld2
idx_Croston = [1, 3]
idx_SBA = [2]


LoadError: BoundsError: attempt to access 1-element Vector{Float64} at index [2]